# Training Loop Refactoring

- Making experimentation easier by using the RunBuilder class and by building another class RunManager which will make it easier to test and experiment with different parameters
    - Clean up TensorBoard calls
- This way it will be easiler to extend and work with different hyperparameters


- Using a RunManager class will allow us finer control over the different hyperparameters under experimenations. The hyperparameters are:
    - Learning rate
    - Batch size
    - Shuffle
- More hyperparameters can be added depending on what is desired
- The RunManager class will conduct a series of runs using all possible combinations of the given hyperparameters. 
- Aside from that, the class will also allow us to save results into a pandas dataframe as well as a csv file for later reference and further analysis

## Class Evolution from DeepLizard.com
<div>
<img src="evolution.png" width="500"/>
</div>

In [28]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms 
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import numpy as np
import pandas as pd
import json

import time

import matplotlib.pyplot as plt
import seaborn as sns

from torch.utils.tensorboard import SummaryWriter

from IPython.display import clear_output



from collections import OrderedDict
from collections import namedtuple
from itertools import product

In [14]:
from my_model import Network

In [17]:
# Run Builder 
class RunBuilder():
    @staticmethod
    def get_runs(params):
        
        Run = namedtuple('Run', params.keys())
        
        runs = []
        for v in product(*params.values()):
            runs.append(Run(*v))
        
        return runs
    
# Train set
train_set = torchvision.datasets.FashionMNIST(
    root='./Documents/data'
     ,train=True
    ,download=True # downloads it locally (checks existence beforehand)
    ,transform=transforms.Compose([
        transforms.ToTensor() # butilt in tensor transformer
    ])
)

In [25]:
class RunManager():
    def __init__(self):
        
        # we will need to extract a class out of these epoch values
        self.epoch_count = 0
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        self.epoch_start_time = None
        
        self.run_params = None
        self.run_count = 0
        self.run_data = []
        self.run_start_time = None
        
        self.network = None
        self.loader = None
        self.tb = None
        
    def begin_run(self, run, network, loader):
        
        self.run_start_time = time.time()
        
        self.run_params = run
        self.run_count += 1
        
        self.network = network
        self.loader = loader
        self.tb = SummaryWriter(comment=f'-{run}')
        
        images, labels = next(iter(self.loader))
        grid = torchvision.utils.make_grid(images)
        
    def end_run(self):
        self.tb.close()
        self.epoch_count = 0
        
    def begin_epoch(self):
        self.epoch_start_time = time.time()
        
        self.epoch_count += 1
        self.epoch_loss = 0
        self.epoch_num_correct = 0
        
    def end_epoch(self):
        
        epoch_duration = time.time() - self.epoch_start_time
        run_duration = time.time() - self.run_start_time
        
        loss = self.epoch_loss / len(self.loader.dataset)
        accuracy = self.epoch_num_correct / len(self.loader.dataset)
        
        self.tb.add_scalar('Loss', loss, self.epoch_count)
        self.tb.add_scalar('Accuracy', accuracy, self.epoch_count)
        
        for name, param in self.network.named_parameters():
            self.tb.add_histogram(name, param, self.epoch_count)
            self.tb.add_histogram(f'{name}.grad', param.grad, self.epoch_count)
            
        results = OrderedDict()
        results["run"] = self.run_count
        results["epoch"] = self.epoch_count
        results["loss"] = loss
        results["accuracy"] = accuracy
        results["epoch duration"] = epoch_duration
        results["run duration"] = run_duration
        
        for k,v in self.run_params._asdict().items():
            results[k] = v
        
        self.run_data.append(results)
        
        df = pd.DataFrame.from_dict(self.run_data, orient='columns')
        
        clear_output(wait=True)
        display(df)
        
    def track_loss(self, loss):
        self.epoch_loss += loss.item() * self.loader.batch_size
        
    def track_num_correct(self, preds, labels):
        self.epoch_num_correct += self._get_num_correct(preds, labels)
        
    @torch.no_grad()
    def _get_num_correct(self, preds, labels):
        # underscore indicates that this method should not really 
        # be used outside this class
        return preds.argmax(dim=1).eq(labels).sum().item()
    
    def save(self,fileName):
        
        pd.DataFrame.from_dict(
            self.run_data,
            orient='columns',
        ).to_csv(f'{fileName}.csv')
        
        with open(f'{fileName}.json', 'w', encoding='utf-8') as f:
            json.dump(self.run_data, f, ensure_ascii=False, indent=4)

In [26]:
# Extracting Epoch class
class Epoch():
    def __init__(self):
        self.count = 0
        self.loss = 0
        self.num_correct = 0
        self.start_time = None

## Test RunManager

In [30]:
params = OrderedDict(
    lr = [.01],
    batch_size = [1000, 2000],
    shuffle = [True, False]
)

m = RunManager()

for run in RunBuilder.get_runs(params):
    
    network = Network()
    loader = DataLoader(train_set, batch_size=run.batch_size, shuffle=run.shuffle)
    optimizer = optim.Adam(network.parameters(), lr=run.lr)
    
    m.begin_run(run, network, loader)
    for epoch in range(5):
        m.begin_epoch()
        for batch in loader:
            
            images = batch[0]
            labels = batch[1]
            preds = network(images)
            loss = F.cross_entropy(preds, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            m.track_loss(loss)
            m.track_num_correct(preds, labels)
            
        m.end_epoch()
    m.end_run()
m.save('results')

,run,epoch,loss,accuracy,epoch duration,run duration,lr,batch_size,shuffle
0,1,1,1.123886,0.566567,12.458130,12.579464,0.01,1000,True
1,1,2,0.618371,0.762417,13.371813,26.009580,0.01,1000,True
2,1,3,0.522179,0.805833,13.280519,39.350181,0.01,1000,True
3,1,4,0.449324,0.836000,13.263394,52.674668,0.01,1000,True
4,1,5,0.407587,0.848367,14.465557,67.200300,0.01,1000,True
5,2,1,0.989769,0.623100,14.842460,14.946062,0.01,1000,False
6,2,2,0.517355,0.802850,13.192339,28.229676,0.01,1000,False
7,2,3,0.428460,0.843067,13.336428,41.630929,0.01,1000,False
8,2,4,0.379956,0.859767,13.827180,55.521298,0.01,1000,False
9,2,5,0.353048,0.871200,14.230369,69.814318,0.01,1000,False


# Resources

- https://deeplizard.com/learn/video/ozpv_peZ894